## Merge all normalized data
- As now we have all data normalized we will concat them for our model sake 

In [ ]:
df1=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/feature1N.csv')
df2=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/feature2N.csv')
df3=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/feature5N.csv')
df4=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/feature6N.csv')
df5=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/feature7N.csv')
df6=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/feature8N.csv')



fea=pd.concat([df1,df2,df3,df4,df5,df6],axis=0)
d=fea
x=d['activityID'].unique().tolist()
y=list(range(1,12))

a=d['activityID'].to_frame()
b=a.replace(x,y)
d['activityID']=b

d.to_csv(r'/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/allfnormalized.csv',index=None,header=True)

## before fixing window 
- Output is showing the each activity personwise
- This will be fine for RF and svm


In [12]:
import pandas as pd
df=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/allfnormalized.csv')

m=df['subject'].unique()
for each in m:
    
    data=df.loc[df['subject'] == each,'activityID']
    print(data.value_counts())
    print('\n')



11    379
1     379
8     379
4     371
9     354
10    349
5     343
2     328
6     263
7     233
3     199
Name: activityID, dtype: int64


8     597
9     536
4     479
10    435
3     414
1     340
6     282
5     280
7     235
11    162
2     133
Name: activityID, dtype: int64


4     637
8     553
9     491
11    426
5     417
1     303
10    295
3     248
6     237
7     203
2     145
Name: activityID, dtype: int64


4     730
9     506
8     487
11    411
5     394
1     380
10    370
2     312
3     305
6     224
7     178
Name: activityID, dtype: int64


8     616
4     567
9     549
1     464
3     460
10    414
5     400
6     314
2     206
7     192
11     53
Name: activityID, dtype: int64


4     624
8     568
9     536
10    472
5     452
1     427
3     416
2     414
11    288
6     188
7     152
Name: activityID, dtype: int64




#  <font color='Red'>This preprocessing is for our LSTM based RNN architecture</font> 

## For each subject and for each activity we fix the activity size so that it can be divider by 50 
- First we divide window by 50 and take the mode. if the mode is less then 50/2=25 the we drop the reminder and if is greater than 50/2=25 then we copy from previous window
- Let's say a windo size is 405. we will remove last 5 data to make it 400
- Let's say a window size is 430 . we will add 20 data from index 410-430  to make it 500


In [13]:
import pandas as pd
df=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/allfnormalized.csv')

def insert_loc(oldf,newdf,index):
    prev=oldf[:index+1]
    nextt=oldf.iloc[index+1:]
    
    return pd.concat([prev,newdf,nextt],ignore_index=True,sort=False)
    
    
def drop_loc(oldf,fst,lst):
    df_all=oldf.drop(oldf.index[fst:lst+1])
    df_all = df_all.reset_index(drop=True)
    return df_all




sub=df['subject'].unique()
act=df['activityID'].unique()
l=len(act)
cut=50
size=cut/2
dff=df

for each in sub:
    data=dff.loc[dff['subject'] == each,'activityID']
    
    for i in range(l): 
        value=data.value_counts()[act[i]]
        adf=df.loc[(df['subject'] == each)&(df['activityID']==act[i])]
        last_ind=adf.index[-1]
        mod=value%cut
        
        if(mod>size):
            use=cut-mod
            new=adf.tail(use)
            df=insert_loc(df,new,last_ind)
            
        if(mod<size): 
            first_ind=last_ind-mod+1
            df=drop_loc(df,first_ind,last_ind)
    

m=df['subject'].unique()

for each in m:   
    data=df.loc[df['subject'] == each,'activityID']
    print(data.value_counts())
    print('\n')


11    400
1     400
8     400
9     350
5     350
10    350
4     350
2     350
7     250
6     250
3     200
Name: activityID, dtype: int64


8     600
9     550
4     500
10    450
3     400
1     350
5     300
6     300
7     250
11    150
2     150
Name: activityID, dtype: int64


4     650
8     550
9     500
11    450
5     400
1     300
10    300
3     250
6     250
7     200
2     150
Name: activityID, dtype: int64


4     750
9     500
8     500
11    400
5     400
1     400
10    350
3     300
2     300
7     200
6     200
Name: activityID, dtype: int64


8     600
9     550
4     550
3     450
1     450
10    400
5     400
6     300
7     200
2     200
11     50
Name: activityID, dtype: int64


4     600
9     550
8     550
10    450
5     450
1     450
3     400
2     400
11    300
6     200
7     150
Name: activityID, dtype: int64




## Write this format in another csv

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('/home/kishore/Datasets/PAMAP2_Dataset/cleaned/new/total.csv')